This file allows for testing the GraphWorld setup with GNN implementations.
It is currently set up to test the SSL methods for the JL benchmarker.

Through this notebook you can attach a debugger.
Note that graph_tool does not work on windows, so we cannot use the graph generators.
Instead, we use the standard datasets from PyG.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from graph_world.self_supervised_learning.benchmarker_jl import NNNodeBenchmarkerJL
from graph_world.models.basic_gnn import GCN, SuperGAT
from torch_geometric.datasets import Planetoid

from graph_world.self_supervised_learning.pretext_tasks.contrastive_based import *


In [5]:
# Parameter setup (for cora)
benchmark_params = {
    'epochs' : 50,
    'lr' : 0.01,
    'lambda' : 1
}

h_params = {
    'in_channels' : 1433,
    'hidden_channels' : 16,
    'num_layers' : 2,
    'dropout' : 0.5,
    'moving_average_decay' : 0.99, 
    'edge_mask_ratio1' : 0.2,
    'edge_mask_ratio2' : 0.4,
    'feature_mask_ratio1' : 0.4, 
    'feature_mask_ratio2' : 0.2
}

generator_config = {
    'num_clusters' : 7,
}

pretext_tasks = [BGRL]

In [3]:
# Get dataset
dataset = Planetoid(root='/tmp/Cora', name='Cora')[0]

In [7]:
# Training. You can attach a debugger to w/e is needed inside train
benchmarker = NNNodeBenchmarkerJL(generator_config=generator_config, model_class=GCN, 
                benchmark_params=benchmark_params, h_params=h_params, pretext_tasks=pretext_tasks)
benchmarker.SetMasks(train_mask=dataset.train_mask, val_mask=dataset.val_mask, test_mask=dataset.test_mask)
benchmarker.train(data=dataset, tuning_metric="rocauc_ovr", tuning_metric_is_loss=False)

GCN(1433, 16, num_layers=2)


c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)
c:\Users\bertr\miniconda3\envs\thesis\lib\site-packages\torch_geometric\deprecation.py:12: User

([6.305573463439941,
  6.295496940612793,
  6.16735315322876,
  6.140897750854492,
  6.070256233215332,
  6.044820785522461,
  5.918725967407227,
  5.846756935119629,
  5.630119800567627,
  5.582365036010742,
  5.423933029174805,
  5.410867214202881,
  5.199756622314453,
  5.028343200683594,
  5.135857582092285,
  5.143563270568848,
  4.95815896987915,
  5.088622570037842,
  4.966188907623291,
  4.88737154006958,
  4.889085292816162,
  4.856661319732666,
  4.710052490234375,
  4.679381847381592,
  4.61484432220459,
  4.573955059051514,
  4.545480728149414,
  4.428616046905518,
  4.3652238845825195,
  4.339372634887695,
  4.3064470291137695,
  4.153972625732422,
  4.049377918243408,
  3.980226993560791,
  3.8427088260650635,
  3.948732852935791,
  3.738022804260254,
  3.7342381477355957,
  3.5954432487487793,
  3.583833694458008,
  3.5478312969207764,
  3.5001935958862305,
  3.443312883377075,
  3.385009288787842,
  3.3414859771728516,
  3.268446445465088,
  3.282491683959961,
  3.11286